# Data Preprocessing for NER Dataset

This notebook handles data preprocessing including:
- Sentence reconstruction
- Vocabulary building
- Sequence encoding and padding
- Train/validation/test splits

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import pickle
from collections import Counter

# Add src directory to path
sys.path.append('../src')

from data_preprocessing import NERDataProcessor, analyze_dataset
from utils import (
    load_dataset, get_unique_tags, save_results,
    print_dataset_info, plot_tag_distribution
)

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

## 1. Load and Inspect Data

In [ ]:
# Load the dataset
data_path = '../data/ner_dataset.csv'
df = load_dataset(data_path)

print(f"Dataset loaded: {df.shape[0]:,} tokens across {df['Sentence #'].nunique():,} sentences")
print(f"Unique words: {df['Word'].nunique():,}")
print(f"Unique tags: {df['Tag'].nunique()}")

# Show sample
df.head()

## 2. Initialize Data Processor

In [ ]:
# Initialize the data processor
# Set max_sequence_length based on sentence length analysis
sentence_lengths = df.groupby('Sentence #').size()
max_length = int(sentence_lengths.quantile(0.95))  # Use 95th percentile

print(f"Sentence length statistics:")
print(f"Mean: {sentence_lengths.mean():.1f}")
print(f"Median: {sentence_lengths.median():.1f}")
print(f"95th percentile: {max_length}")
print(f"Max: {sentence_lengths.max()}")

# Initialize processor with the calculated max length
processor = NERDataProcessor(max_sequence_length=max_length)
print(f"\nInitialized NERDataProcessor with max_sequence_length={max_length}")

## 3. Process Data

In [ ]:
# Process the complete dataset
print("Processing dataset...")
processed_data = processor.process_data(data_path)

print("\nProcessing completed!")
print(f"Vocabulary size: {processed_data['metadata']['vocab_size']:,}")
print(f"Number of tags: {processed_data['metadata']['num_tags']}")
print(f"Max sequence length: {processed_data['metadata']['max_sequence_length']}")

# Extract data splits
X_train = processed_data['X_train']
X_val = processed_data['X_val']
X_test = processed_data['X_test']
y_train = processed_data['y_train']
y_val = processed_data['y_val']
y_test = processed_data['y_test']
metadata = processed_data['metadata']

print(f"\nData splits:")
print(f"Training: {X_train.shape[0]:,} sequences")
print(f"Validation: {X_val.shape[0]:,} sequences")
print(f"Test: {X_test.shape[0]:,} sequences")

## 4. Examine Processed Data

In [ ]:
# Examine the shape and properties of processed data
print("Training data shapes:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")

print(f"\nSample input sequence (first 10 tokens):")
print(X_train[0][:10])

print(f"\nSample output sequence (first 10 tags):")
print(y_train[0][:10])

# Convert back to readable format for inspection
id_to_tag = metadata['id_to_tag']
sample_tags = [id_to_tag[str(tag_id)] for tag_id in y_train[0][:20]]
print(f"\nSample tags (readable): {sample_tags}")

In [ ]:
# Examine tag distribution in processed data
tag_to_id = metadata['tag_to_id']
print("Tag mappings:")
for tag, tag_id in sorted(tag_to_id.items(), key=lambda x: x[1]):
    print(f"{tag}: {tag_id}")

# Calculate tag distribution in training data
train_tag_counts = Counter(y_train.flatten())
print("\nTag distribution in training data:")
for tag_id, count in sorted(train_tag_counts.items()):
    tag_name = id_to_tag[str(tag_id)]
    percentage = (count / len(y_train.flatten())) * 100
    print(f"{tag_name} (ID {tag_id}): {count:,} ({percentage:.2f}%)")

## 5. Analyze Sequence Properties

In [ ]:
# Analyze padding in sequences
pad_token_id = 0  # Assuming PAD token has ID 0

def count_non_pad_tokens(sequences, pad_id=0):
    """Count non-padding tokens in each sequence."""
    return [(seq != pad_id).sum() for seq in sequences]

train_lengths = count_non_pad_tokens(X_train)
val_lengths = count_non_pad_tokens(X_val)
test_lengths = count_non_pad_tokens(X_test)

print("Actual sequence lengths (excluding padding):")
print(f"Training - Mean: {np.mean(train_lengths):.1f}, Max: {np.max(train_lengths)}, Min: {np.min(train_lengths)}")
print(f"Validation - Mean: {np.mean(val_lengths):.1f}, Max: {np.max(val_lengths)}, Min: {np.min(val_lengths)}")
print(f"Test - Mean: {np.mean(test_lengths):.1f}, Max: {np.max(test_lengths)}, Min: {np.min(test_lengths)}")

# Plot sequence length distribution
plt.figure(figsize=(12, 6))
plt.hist(train_lengths, bins=50, alpha=0.7, label='Training', density=True)
plt.hist(val_lengths, bins=50, alpha=0.7, label='Validation', density=True)
plt.hist(test_lengths, bins=50, alpha=0.7, label='Test', density=True)
plt.xlabel('Actual Sequence Length')
plt.ylabel('Density')
plt.title('Distribution of Actual Sequence Lengths')
plt.legend()
plt.axvline(max_length, color='red', linestyle='--', label=f'Max Length ({max_length})')
plt.tight_layout()
plt.show()

In [ ]:
# Check how much data is truncated
truncated_count = sum(1 for length in sentence_lengths if length > max_length)
truncation_percentage = (truncated_count / len(sentence_lengths)) * 100

print(f"Sequences longer than max_length ({max_length}): {truncated_count:,} ({truncation_percentage:.2f}%)")
print(f"Sequences kept fully: {len(sentence_lengths) - truncated_count:,} ({100 - truncation_percentage:.2f}%)")

if truncation_percentage > 5:
    print("⚠️  Warning: More than 5% of sequences are truncated. Consider increasing max_sequence_length.")
else:
    print("✅ Good: Less than 5% of sequences are truncated.")

## 6. Save Processed Data

In [ ]:
# Create directories for saving
os.makedirs('../models', exist_ok=True)
os.makedirs('../results', exist_ok=True)

# Save the data processor
processor_path = '../models/data_processor.pkl'
processor.save_processor(processor_path)
print(f"Data processor saved to: {processor_path}")

# Save processed data
data_save_path = '../results/processed_data.npz'
np.savez_compressed(
    data_save_path,
    X_train=X_train,
    X_val=X_val,
    X_test=X_test,
    y_train=y_train,
    y_val=y_val,
    y_test=y_test
)
print(f"Processed data saved to: {data_save_path}")

# Save metadata
metadata_path = '../results/preprocessing_metadata.json'
save_results(metadata, metadata_path)
print(f"Metadata saved to: {metadata_path}")

## 7. Validation and Quality Checks

In [ ]:
# Validate data integrity
print("Data Validation:")
print("=" * 20)

# Check shapes
assert X_train.shape[0] == y_train.shape[0], "Training X and y have different number of samples"
assert X_val.shape[0] == y_val.shape[0], "Validation X and y have different number of samples"
assert X_test.shape[0] == y_test.shape[0], "Test X and y have different number of samples"
print("✅ Shape consistency check passed")

# Check sequence lengths
assert X_train.shape[1] == max_length, "Training sequences have wrong length"
assert y_train.shape[1] == max_length, "Training labels have wrong length"
print("✅ Sequence length check passed")

# Check value ranges
assert X_train.min() >= 0, "Negative values in X_train"
assert X_train.max() < metadata['vocab_size'], "X_train values exceed vocab size"
assert y_train.min() >= 0, "Negative values in y_train"
assert y_train.max() < metadata['num_tags'], "y_train values exceed num_tags"
print("✅ Value range check passed")

# Check data splits don't overlap
total_samples = X_train.shape[0] + X_val.shape[0] + X_test.shape[0]
expected_samples = metadata['total_sentences']
assert total_samples == expected_samples, f"Data split error: {total_samples} != {expected_samples}"
print("✅ Data split check passed")

print("\n🎉 All validation checks passed!")

In [ ]:
# Test the processor with a sample text
sample_text = "Barack Obama visited New York yesterday."
print(f"Testing processor with sample text: '{sample_text}'")

# Preprocess the text
processed_sample = processor.preprocess_text(sample_text)
print(f"Processed shape: {processed_sample.shape}")
print(f"Processed sequence (first 10 tokens): {processed_sample[0][:10]}")

# Show the word mappings for this text
words = sample_text.split()
word_ids = [processor.word_to_id.get(word, processor.word_to_id['<UNK>']) for word in words]
print(f"\nWord to ID mapping:")
for word, word_id in zip(words, word_ids):
    status = "(UNK)" if word_id == processor.word_to_id['<UNK>'] else ""
    print(f"  {word}: {word_id} {status}")

## 8. Summary Statistics

In [ ]:
# Create summary statistics
summary_stats = {
    'dataset_info': {
        'total_tokens': int(df.shape[0]),
        'total_sentences': int(metadata['total_sentences']),
        'vocabulary_size': int(metadata['vocab_size']),
        'num_tags': int(metadata['num_tags']),
        'max_sequence_length': int(metadata['max_sequence_length'])
    },
    'data_splits': {
        'train_samples': int(X_train.shape[0]),
        'val_samples': int(X_val.shape[0]),
        'test_samples': int(X_test.shape[0]),
        'train_percentage': round((X_train.shape[0] / total_samples) * 100, 1),
        'val_percentage': round((X_val.shape[0] / total_samples) * 100, 1),
        'test_percentage': round((X_test.shape[0] / total_samples) * 100, 1)
    },
    'sequence_stats': {
        'mean_train_length': round(np.mean(train_lengths), 1),
        'mean_val_length': round(np.mean(val_lengths), 1),
        'mean_test_length': round(np.mean(test_lengths), 1),
        'truncation_percentage': round(truncation_percentage, 2)
    },
    'tag_distribution': {id_to_tag[str(tag_id)]: int(count) for tag_id, count in train_tag_counts.items()}
}

# Save summary statistics
summary_path = '../results/preprocessing_summary.json'
save_results(summary_stats, summary_path)
print(f"Summary statistics saved to: {summary_path}")

# Display summary
print("\nPreprocessing Summary:")
print("=" * 30)
print(f"📊 Dataset: {summary_stats['dataset_info']['total_tokens']:,} tokens, {summary_stats['dataset_info']['total_sentences']:,} sentences")
print(f"📚 Vocabulary: {summary_stats['dataset_info']['vocabulary_size']:,} unique words")
print(f"🏷️  Tags: {summary_stats['dataset_info']['num_tags']} unique NER tags")
print(f"📏 Max Length: {summary_stats['dataset_info']['max_sequence_length']} tokens")
print(f"\n🔄 Data Splits:")
print(f"   • Training: {summary_stats['data_splits']['train_samples']:,} ({summary_stats['data_splits']['train_percentage']}%)")
print(f"   • Validation: {summary_stats['data_splits']['val_samples']:,} ({summary_stats['data_splits']['val_percentage']}%)")
print(f"   • Test: {summary_stats['data_splits']['test_samples']:,} ({summary_stats['data_splits']['test_percentage']}%)")
print(f"\n✂️ Truncation: {summary_stats['sequence_stats']['truncation_percentage']}% of sequences truncated")

## Conclusion

Data preprocessing completed successfully!

**What we accomplished:**
1. ✅ Loaded and analyzed the raw NER dataset
2. ✅ Built vocabulary mappings for words and tags
3. ✅ Converted text sequences to numerical representations
4. ✅ Applied appropriate padding/truncation
5. ✅ Split data into train/validation/test sets (60%/20%/20%)
6. ✅ Validated data integrity and quality
7. ✅ Saved processed data and metadata for model training

**Next steps:**
- The processed data is ready for model training
- Both baseline and advanced models can use this preprocessed data
- All mappings and metadata are preserved for consistent evaluation